# MultiModulon Analysis Demo

This notebook demonstrates the complete workflow for multi-species/strain modular analysis using the MultiModulon package.

## Setup and Imports

In [ ]:
# Import required libraries
from multimodulon import MultiModulon
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

print("Setup complete!")

## Step 1: Initialize MultiModulon

Load data from the Input_Data directory containing expression matrices, gene annotations, and sample metadata for all strains.

In [ ]:
# Path to the Input_Data folder
input_data_path = '../imminer_2_industrial_strain/Input_Data'

# Initialize MultiModulon object
multiModulon = MultiModulon(input_data_path)

# Print summary
multiModulon.summary()

In [ ]:
# Explore available strains
available_strains = multiModulon.species
print(f"Available strains: {available_strains}")
print(f"Number of strains: {len(available_strains)}")

In [ ]:
# Examine data for the first strain
first_strain = available_strains[0]
strain_data = multiModulon[first_strain]

print(f"\n{first_strain} data:")
print(f"  - Log TPM matrix shape: {strain_data.log_tpm.shape}")
print(f"  - Log TPM normalized matrix shape: {strain_data.log_tpm_norm.shape}")
print(f"  - Sample sheet shape: {strain_data.sample_sheet.shape}")
print(f"\nFirst 5 genes:")
print(strain_data.log_tpm.index[:5].tolist())
print(f"\nFirst 5 samples:")
print(strain_data.log_tpm.columns[:5].tolist())

## Step 2: Generate BBH Files

Generate Bidirectional Best Hits (BBH) files for ortholog detection between all strain pairs.

In [ ]:
# Generate BBH files using multiple threads for faster computation
output_bbh_path = '../imminer_2_industrial_strain/Output_BBH'

print("Generating BBH files...")
print("This may take several minutes depending on the number of strains and genes.")

multiModulon.generate_BBH(output_bbh_path, threads=8)

## Step 3: Align Genes Across Strains

Create a unified gene database by aligning genes across all strains using the BBH results.

In [ ]:
# Align genes across all strains
output_gene_info_path = '../imminer_2_industrial_strain/Output_Gene_Info'

combined_gene_db = multiModulon.align_genes(
    input_bbh_dir=output_bbh_path,
    output_dir=output_gene_info_path,
    reference_order=['MG1655', 'BL21', 'C', 'Crooks', 'W', 'W3110'],  # optional: specify order
    bbh_threshold=90  # optional: minimum percent identity threshold
)

print(f"\nCombined gene database shape: {combined_gene_db.shape}")
print(f"Number of gene groups: {len(combined_gene_db)}")
print(f"\nFirst 5 gene groups:")
combined_gene_db.head()

## Step 4: Create Gene Tables

Parse GFF files to create gene annotation tables for each strain.

In [ ]:
# Create gene tables from GFF files
print("Creating gene tables from GFF files...")
multiModulon.create_gene_table()

# Check gene tables for each strain
for strain_name in available_strains[:3]:  # Show first 3 strains
    strain_data = multiModulon[strain_name]
    if strain_data.gene_table is not None:
        print(f"\n{strain_name} gene table:")
        print(f"  Shape: {strain_data.gene_table.shape}")
        print(f"  Columns: {list(strain_data.gene_table.columns)}")
        print(f"\n  First 3 genes:")
        display(strain_data.gene_table.head(3))

## Step 5: Generate Aligned Expression Matrices

Create expression matrices with consistent gene indexing across all strains for multi-view ICA.

In [ ]:
# Generate aligned expression matrices
print("Generating aligned expression matrices...")
multiModulon.generate_X(output_gene_info_path)

# The output shows aligned X matrices and dimension recommendations

In [ ]:
# Check aligned matrices
print("\nAligned X matrices:")
for strain_name in available_strains:
    X = multiModulon[strain_name].X
    non_zero = (X != 0).any(axis=1).sum()
    print(f"{strain_name}: {X.shape} ({non_zero} non-zero gene groups)")

# Verify that all matrices have the same gene index
first_index = multiModulon[available_strains[0]].X.index
all_same = all(multiModulon[strain].X.index.equals(first_index) for strain in available_strains)
print(f"\nAll matrices have the same gene index: {all_same}")

## Step 6: Optimize Number of Core Components

Use Cohen's d effect size metric to automatically determine the optimal number of core (shared) components.

In [ ]:
# Optimize number of core components
print("Optimizing number of core components...")
print("This will test different values of k and find the optimal number.")

optimal_num_core_components = multiModulon.optimize_number_of_core_components(
    metric='effect_size',          # Use Cohen's d effect size metric
    effect_size_threshold=5,       # Components must have Cohen's d > 5
    step=5,                        # Test k = 5, 10, 15, 20, ...
    save_path='optimization_plots', # Save plots to directory
    fig_size=(5, 3),              # Figure size
    num_runs=1                     # Number of runs (increase for more robust results)
)

print(f"\nOptimal number of core components: {optimal_num_core_components}")

In [ ]:
# Display the optimization plot
from IPython.display import Image, display
import os

if os.path.exists('optimization_plots/num_core_optimization.svg'):
    print("Core component optimization plot saved to: optimization_plots/num_core_optimization.svg")
    # Note: SVG display in Jupyter might require conversion to PNG

## Step 7: Optimize Number of Unique Components

Determine the optimal number of unique (species-specific) components for each strain.

In [ ]:
# Optimize unique components for each species
print("Optimizing unique components per species...")
print("This will test different numbers of unique components for each species.\n")

optimal_unique, optimal_total = multiModulon.optimize_number_of_unique_components(
    optimal_num_core_components=optimal_num_core_components,
    step=5,
    save_path='optimization_plots',
    fig_size=(5, 3)
)

print("\nOptimization complete!")
print("\nOptimal unique components per species:")
for species, n_unique in optimal_unique.items():
    print(f"  {species}: {n_unique} unique components")

print("\nOptimal total components per species:")
for species, n_total in optimal_total.items():
    print(f"  {species}: {n_total} total components (core + unique)")

## Step 8: Run Robust Multi-view ICA

Perform robust multi-view ICA with multiple runs and clustering to identify consistent components.

In [ ]:
# Run robust multi-view ICA
print("Running robust multi-view ICA with clustering...")
print("This performs multiple ICA runs and clusters the results for robustness.\n")

M_matrices, A_matrices = multiModulon.run_robust_multiview_ica(
    a=optimal_total,               # Dictionary of total components per species
    c=optimal_num_core_components, # Number of core components
    num_runs=100,                  # Number of runs for robustness
    effect_size_threshold=5,       # Cohen's d threshold
    seed=42                       # Random seed for reproducibility
)

print("\nRobust ICA complete!")

In [ ]:
# Check ICA results
print("ICA results summary:")
print("=" * 60)

for species_name in available_strains:
    M = multiModulon[species_name].M
    A = multiModulon[species_name].A
    
    if M is not None and A is not None:
        # Count core and unique components
        n_core = len([c for c in M.columns if c.startswith('Core_')])
        n_unique = len([c for c in M.columns if c.startswith('Unique_')])
        
        print(f"\n{species_name}:")
        print(f"  M matrix (genes × components): {M.shape}")
        print(f"  A matrix (components × samples): {A.shape}")
        print(f"  Components: {n_core} core, {n_unique} unique")
        
        # Show component names
        print(f"  Core components: {[c for c in M.columns if c.startswith('Core_')][:5]}...")
        if n_unique > 0:
            print(f"  Unique components: {[c for c in M.columns if c.startswith('Unique_')][:5]}...")

## Step 9: Calculate Explained Variance

Assess how much of the expression variance is explained by the ICA components.

In [ ]:
# Calculate explained variance
print("Calculating explained variance for each species...\n")

explained_var = multiModulon.calculate_explained_variance()

# Create a bar plot of explained variance
plt.figure(figsize=(8, 5))
species_names = list(explained_var.keys())
variances = list(explained_var.values())

bars = plt.bar(species_names, variances, color='skyblue', edgecolor='navy')
plt.xlabel('Species/Strain', fontsize=12)
plt.ylabel('Explained Variance', fontsize=12)
plt.title('Explained Variance by Species', fontsize=14, fontweight='bold')
plt.ylim(0, 1.0)

# Add value labels on bars
for bar, var in zip(bars, variances):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{var:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Print summary
print("\nExplained variance summary:")
for species, variance in explained_var.items():
    print(f"  {species}: {variance:.4f} ({variance*100:.2f}%)")

avg_variance = np.mean(list(explained_var.values()))
print(f"\nAverage explained variance: {avg_variance:.4f} ({avg_variance*100:.2f}%)")

## Step 10: Visualize iModulon Weights

Create visualizations showing how gene weights are distributed across the genome for specific components.

In [ ]:
# Visualize a core component for one species
species_to_plot = 'MG1655' if 'MG1655' in available_strains else available_strains[0]
core_component = 'Core_1'

print(f"Visualizing {core_component} for {species_to_plot}...")

multiModulon.view_iModulon_weights(
    species=species_to_plot,
    component=core_component,
    save_path='imodulon_plots',
    fig_size=(8, 5)
)

print(f"Plot saved to: imodulon_plots/{species_to_plot}_{core_component}_iModulon.svg")

In [ ]:
# Visualize a unique component (if available)
unique_components = [c for c in multiModulon[species_to_plot].M.columns if c.startswith('Unique_')]

if unique_components:
    unique_component = unique_components[0]
    print(f"\nVisualizing {unique_component} for {species_to_plot}...")
    
    multiModulon.view_iModulon_weights(
        species=species_to_plot,
        component=unique_component,
        save_path='imodulon_plots',
        fig_size=(8, 5)
    )
    
    print(f"Plot saved to: imodulon_plots/{species_to_plot}_{unique_component}_iModulon.svg")
else:
    print(f"\nNo unique components found for {species_to_plot}")

In [ ]:
# Visualize a core component across all species
print(f"\nVisualizing {core_component} across all species...")

multiModulon.view_core_iModulon_weights(
    component=core_component,
    save_path='imodulon_plots',
    fig_size=(6, 4)
)

print(f"\nGenerated plots for {core_component} across all species in: imodulon_plots/")

## Step 11: Explore Component Details

Examine the top genes in specific components to understand their biological functions.

In [ ]:
# Function to get top genes for a component
def get_top_genes(species, component, n_top=20):
    """Get the top weighted genes for a specific component."""
    M = multiModulon[species].M
    gene_table = multiModulon[species].gene_table
    
    # Get component weights
    weights = M[component].abs().sort_values(ascending=False)
    top_genes = weights.head(n_top)
    
    # Create results dataframe
    results = pd.DataFrame({
        'gene': top_genes.index,
        'weight': M.loc[top_genes.index, component].values,
        'abs_weight': top_genes.values
    })
    
    # Add gene annotations if available
    if gene_table is not None:
        # Get annotations for genes that exist in gene_table
        annotations = []
        for gene in results['gene']:
            if gene in gene_table.index:
                gene_info = gene_table.loc[gene]
                annotation = gene_info.get('product', 'N/A')
                annotations.append(annotation)
            else:
                annotations.append('N/A')
        results['annotation'] = annotations
    
    return results

# Example: Get top genes for Core_1
species_example = 'MG1655' if 'MG1655' in available_strains else available_strains[0]
top_genes_core1 = get_top_genes(species_example, 'Core_1', n_top=15)

print(f"Top 15 genes in {core_component} for {species_example}:")
display(top_genes_core1)

In [ ]:
# Compare Core_1 across multiple species
print(f"Comparing top genes in {core_component} across species:\n")

n_species_to_compare = min(3, len(available_strains))
for species in available_strains[:n_species_to_compare]:
    print(f"\n{species}:")
    top_genes = get_top_genes(species, core_component, n_top=10)
    print(top_genes[['gene', 'weight', 'abs_weight']].to_string(index=False))

## Step 12: Save Analysis Results

Save the complete MultiModulon object for future use.

In [ ]:
# Save the entire analysis
save_filename = 'multimodulon_analysis_complete.json'

print(f"Saving complete analysis to {save_filename}...")
multiModulon.save_to_json_multimodulon(save_filename)
print("✓ Analysis saved successfully!")

# Demonstrate loading
print("\nTesting load functionality...")
loaded_multiModulon = MultiModulon.load_json_multimodulon(save_filename)
print(f"✓ Successfully loaded {len(loaded_multiModulon._species_data)} species from JSON")

# Verify loaded data
print("\nVerifying loaded data:")
for species in available_strains[:2]:
    original_M = multiModulon[species].M
    loaded_M = loaded_multiModulon[species].M
    if original_M is not None and loaded_M is not None:
        is_equal = original_M.equals(loaded_M)
        print(f"  {species} M matrix matches: {is_equal}")

## Summary

This notebook demonstrated the complete MultiModulon workflow:

1. **Data Loading**: Initialized MultiModulon with expression data from multiple strains
2. **BBH Generation**: Created ortholog mappings between all strain pairs
3. **Gene Alignment**: Built a unified gene database across strains
4. **Data Preparation**: Generated aligned expression matrices for ICA
5. **Optimization**: Found optimal numbers of core and unique components
6. **Robust ICA**: Performed multi-view ICA with clustering for robustness
7. **Analysis**: Calculated explained variance and explored component content
8. **Visualization**: Created plots showing gene weight distributions
9. **Persistence**: Saved and loaded the complete analysis

### Generated Files:
- BBH files: `Output_BBH/`
- Gene alignment: `Output_Gene_Info/combined_gene_db.csv`
- Optimization plots: `optimization_plots/`
- ICA clustering plots: `ica_clustering_plots/`
- iModulon visualizations: `imodulon_plots/`
- Complete analysis: `multimodulon_analysis_complete.json`

### Next Steps:
- Explore biological functions of identified components
- Compare component activities across conditions
- Integrate with additional omics data
- Perform enrichment analysis on component genes

In [ ]:
# Final summary statistics
print("Analysis Summary Statistics:")
print("=" * 50)
print(f"Number of strains analyzed: {len(available_strains)}")
print(f"Number of gene groups: {len(combined_gene_db)}")
print(f"Optimal core components: {optimal_num_core_components}")
print(f"\nComponents per strain:")
for species, total in optimal_total.items():
    unique = optimal_unique[species]
    print(f"  {species}: {total} total ({optimal_num_core_components} core + {unique} unique)")

print(f"\nAverage explained variance: {avg_variance:.3f}")
print("\n✓ Analysis complete!")